In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')

from skimage.io import imread, imshow
from skimage.transform import resize

import tensorflow as tf
import gc

import pickle
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input , Dense, Flatten, Dropout, Conv2D, MaxPool2D

from tensorflow.keras.applications import inception_resnet_v2

import tensorflow_datasets as tfds
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import TopKCategoricalAccuracy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
resnet = inception_resnet_v2.InceptionResNetV2()

225222656/225209952 [==============================] - 2s 0us/step


In [4]:
IMG_LEN = 299
IMG_SHAPE = (IMG_LEN, IMG_LEN, 3)
N_BREEDS = 60

In [5]:
opener = open(f'drive/MyDrive/1000_pics.pickle', 'rb')
images = pickle.load(opener)
opener.close()

In [6]:
opener = open(f'drive/MyDrive/1000_labels.pickle', 'rb')
labels = pickle.load(opener)
opener.close()

In [7]:
resnet = Model(inputs = resnet.layers[0].input, outputs = resnet.layers[780].output)

In [8]:
custom_model = Sequential([
                           resnet,
                           Dense(N_BREEDS, activation = 'softmax')
])

In [9]:
custom_model.compile(optimizer = Adam(learning_rate = 0.00001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [10]:
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 7)

In [11]:
gc.collect()

217

In [12]:
!kill process_id

/bin/bash: line 0: kill: process_id: arguments must be process or job IDs


In [16]:
perm = tf.random.shuffle(tf.range(tf.shape(images)[0]))
images = tf.gather(images, perm, axis=0)
labels = tf.gather(labels, perm, axis=0)

In [18]:
history = custom_model.fit(images, labels, validation_split = 0.2, epochs = 50 , callbacks = [early_stopping])

Epoch 1/50
25/25 [==============================] - 29s 1s/step - loss: 1.9851 - accuracy: 0.8400 - val_loss: 2.0601 - val_accuracy: 0.7350
Epoch 2/50
25/25 [==============================] - 29s 1s/step - loss: 1.5535 - accuracy: 0.8750 - val_loss: 1.6415 - val_accuracy: 0.7700
Epoch 3/50
25/25 [==============================] - 29s 1s/step - loss: 1.2104 - accuracy: 0.9062 - val_loss: 1.3143 - val_accuracy: 0.8050
Epoch 4/50
25/25 [==============================] - 29s 1s/step - loss: 0.9187 - accuracy: 0.9287 - val_loss: 1.0767 - val_accuracy: 0.8200
Epoch 5/50
25/25 [==============================] - 29s 1s/step - loss: 0.6908 - accuracy: 0.9475 - val_loss: 0.9254 - val_accuracy: 0.8150
Epoch 6/50
25/25 [==============================] - 29s 1s/step - loss: 0.5270 - accuracy: 0.9700 - val_loss: 0.8082 - val_accuracy: 0.8300
Epoch 7/50
25/25 [==============================] - 29s 1s/step - loss: 0.3903 - accuracy: 0.9787 - val_loss: 0.7432 - val_accuracy: 0.8350
Epoch 8/50
25/25 [==

In [19]:
custom_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 1536)              54336736  
                                                                 
 dense (Dense)               (None, 60)                92220     
                                                                 
Total params: 54,428,956
Trainable params: 54,368,412
Non-trainable params: 60,544
_________________________________________________________________


In [20]:
custom_model.save("drive/MyDrive/model_comp.h5")